## TÖL403G Lokaverkefni

In [98]:
import random, time, math
from typing import NamedTuple
from heapdict import heapdict
from ast import literal_eval
from copy import deepcopy

Byrjum á því að útfæra einfalda gagnagrind fyrir net. Þetta mun halda utan um upplýsingarnar í skránum ásamt því að gera okkur kleift að útfæra aðferðir sem munu vera gagnlegar síðar.

In [61]:
class Node(NamedTuple):
    id: int
    x_coord: float
    y_coord: float
    primary: bool

class Edge(NamedTuple):
    node_id_from: int
    node_id_to: int
    length: float
    name: list[str]


class Graph:

    def __init__(self):
        self.nodes: dict[int, Node] = {}
        self.edges: dict[tuple[int, int], list[Edge]] = {}
        self.adj_list: dict[int, list[int]] = {}

    def add_node(self, node: Node) -> None:
        id = node.id
        if id not in self.nodes:
            self.nodes[id] = node

    def add_edge(self, edge: Edge) -> None:
        id_from = edge.node_id_from
        id_to = edge.node_id_to
        if id_from in self.nodes and id_to in self.nodes:
            self.edges.setdefault((id_from, id_to), []).append(edge)
            self.adj_list.setdefault(id_from, []).append(id_to)
        else:
            print(f'skipped edge missing node(s) {id_from} -> {id_to}')


**2.3.1 Þáttun(*)**

Lesið inn netið úr skránum sem eru gefnar, nodes.tsv og edges.tsv. Í skránni nodes.tsv
eru hnútar með auðkenni (id), hnit (x, y) og hvort þeir séu á aðalvegi (primary). Í skránni
edges.tsv eru leggi frá hnúti u til hnúts v með lengd/length, mæld í metrum, og nafn
(name).

In [62]:
graph = Graph()

with open('nodes.tsv', 'r', encoding='utf-8') as file:
    next(file) # skip header
    for line in file:
        parts = line.strip().split('\t')
        node = Node(
            id = int(parts[0]),
            x_coord = float(parts[1]),
            y_coord = float(parts[2]),
            primary = parts[3].lower() == 'true'
        )
        graph.add_node(node)

def parse_edge_name(name: str) -> list[str]:
    name = name.strip()
    if not name:
        return []
    
    if name.startswith('[') and name.endswith(']'):
        try:
            result = literal_eval(name)
            if isinstance(result, list[str]):
                return result
            else:
                return [str(result)]
        except Exception:
            return [name]
    return [name]

with open('edges.tsv', 'r', encoding='utf-8') as file:
    next(file) # skip header
    for line in file:
        parts = line.strip().split('\t')
        edge = Edge(
            node_id_from = int(parts[0]),
            node_id_to = int(parts[1]),
            length = float(parts[2]),
            name = parse_edge_name(parts[3]) if len(parts) > 3 else []
        )
        graph.add_edge(edge)

***2.3.2 Leit (⋆⋆)***

Ef við setjum hleðslustöðvar á hnúta $v_1, . . . , v_k$ þá er hægt að nota reikniriti Dijkstra til að
finna stystu fjarlægð frá hverjum hnúti $u$ í hleðslustöð $v_i$. Útfærið reikniritið sem tekur
inn lista af lokahnútum og reiknar fjarlægðir frá öllum hnútum í netinu. Athugið að netið
er stefnt net.

In [ ]:

def multisource_dijkstra(g: Graph, source_ids: list[int], target_id: int = None) -> tuple[dict[int, float], dict[int, int]]:
    if not g.nodes:
        raise ValueError('Graph is empty')
    for s_id in source_ids:
        if s_id not in g.nodes:
            raise ValueError(f'source id: {s_id} not in graph')
    if target_id is not None and target_id not in g.nodes:
        raise ValueError(f'target id: {target_id} not in graph')

    INF = float('inf')
    dist = {}
    prev = {}
    Q = heapdict()

    for s_id in source_ids:
        dist[s_id] = 0
        Q[s_id] = 0

    for v in g.nodes:
        if v not in Q:
            dist[v] = INF
            prev[v] = None
            Q[v] = INF

    while Q:
        u, _ = Q.popitem()
        if target_id is not None and u == target_id:
            break

        for neighbour in g.adj_list.get(u, []):
            edge_len = min([edge.length for edge in g.edges.get((u, neighbour), [])]) # sometimes multiple edges, we will always choose the min(length) edge
            if neighbour in Q:
                alt = dist[u] + edge_len
                if alt < dist[neighbour]:
                    dist[neighbour] = alt
                    prev[neighbour] = u
                    Q[neighbour] = alt

    return (dist, prev)

In [ ]:
def Dijkstra(g: Graph, source_id: int, target_id: int = None) -> tuple[dict[int, float], dict[int, int]]:
    if not g.nodes:
        raise ValueError('Graph is empty')
    if source_id not in g.nodes:
        raise ValueError(f'source id: {source_id} not in graph')
    if target_id is not None and target_id not in g.nodes:
        raise ValueError(f'target id: {target_id} not in graph')

    INF = float('inf')
    dist = {}
    prev = {}
    Q = heapdict()
    dist[source_id] = 0
    Q[source_id] = dist[source_id]
    for v, _ in g.nodes.items():
        if v != source_id:
            dist[v] = INF
            prev[v] = None
            Q[v] = dist[v]

    while Q:
        u, _ = Q.popitem()
        if target_id is not None and u == target_id:
            break

        for neighbour in g.adj_list.get(u, []):
            for edge in g.edges.get((u, neighbour), []):
                if neighbour in Q:
                    alt = dist[u] + edge.length
                    if alt < dist[neighbour]:
                        dist[neighbour] = alt
                        prev[neighbour] = u
                        Q[neighbour] = dist[neighbour]

    return (dist, prev)


def reverse_graph(g: Graph) -> Graph:
    reversed_graph = Graph()
    reversed_graph.nodes = g.nodes.copy()

    for _, edges in g.edges.items():
        for edge in edges:
            reversed_edge = Edge(
                node_id_from = edge.node_id_to,
                node_id_to = edge.node_id_from,
                length = edge.length,
                name = edge.name
            )
            reversed_graph.add_edge(reversed_edge)
    
    return reversed_graph

def distances_to_charging_stations_with_Dijkstra(g: Graph, V: list[int]) -> dict[dict[int, float]]:
    '''
    Finds distances to all nodes u in graph from a list of node ID's V
    '''
    g_reversed = reverse_graph(g)
    distances = {}
    for v in V:
        (dist, _) = Dijkstra(g_reversed, v)
        distances[v] = dist
    return distances


***2.3.3 Framsetning (⋆)***

Setjið fimm hleðslustöðvar í netið og sýnið stystu leið fyrir fimm punkta og teiknið upp á
kort. Tékkið ykkur af með því að bera saman leiðina sem er fundin og fjarlægðina miðað
við kortavefi eins og t.d. Google Maps.

In [ ]:

# Multisource Dijsktra!?
import folium

# Center the map near your graph area
m = folium.Map(location=[64.1355, -21.8954], zoom_start=14)  # Use your coords here

# Example nodes with real coordinates
nodes = {
    1: (64.1355, -21.8954),
    2: (64.1362, -21.8922),
    3: (64.1340, -21.8930)
}

# Add nodes to the map
for node_id, (lat, lon) in nodes.items():
    folium.CircleMarker(
        location=(lat, lon),
        radius=6,
        popup=f"Node {node_id}",
        color='blue',
        fill=True,
        fill_opacity=0.7
    ).add_to(m)

# Add edges with labels (e.g. street names)
edges = [
    (1, 2, "Maple St"),
    (2, 3, "Oak Ave"),
    (1, 3, "Pine Rd")
]

for u, v, street_name in edges:
    coords = [nodes[u], nodes[v]]
    folium.PolyLine(
        coords,
        color='green',
        weight=4,
        tooltip=street_name
    ).add_to(m)

# Show the map
m

***2.3.4 Tímamælingar (⋆)***

Mælið tímann sem reiknirit Dijkstra tekur að reikna allar fjarlægðir í netinu með fimm
hleðslustöðvum.

In [99]:
charging_stations = random.sample(list(graph.nodes.keys()), 5)

start_time = time.time()
#distances_to_charging_stations_with_Dijkstra(graph, charging_stations)
reversed_graph = reverse_graph(graph)
multisource_dijkstra(reversed_graph, charging_stations)
end_time = time.time()
elapsed_time = end_time - start_time
print(f'{elapsed_time:.3f} sec')

0.225 sec


***2.3.5 $A^*$ reikniritið (⋆⋆)***

Útfærið $A^*$ reikniritið sem tekur inn lista af lokahnútum og reiknar fjarlægðir frá öll-
um hnútum í netinu. Sem neðra mat á fjarlægð á milli hnútanna má taka $d(u, v) =\sqrt{(x_u − x_v )^2 + (y_u − y_v )^2}$, þ.e. beina loftlínu milli punktanna. Mælið tíma og berið saman
við reiknirit Dijkstra.

In [ ]:
def h(u: Node, v: Node) -> float:
    (x_u, y_u) = (u.x_coord, u.y_coord)
    (x_v, y_v) = (v.x_coord, v.y_coord)
    return math.sqrt((x_u - x_v)**2 + (y_u - y_v)**2)

# TODO
# currently appending none and need to add that it calculates path len
def construct_path(prev: dict[int, int], target_id: int) -> list[int]:
    path = [target_id]
    current = target_id
    while current is not None:
        current = prev.get(current, None)
        path.append(current)

    path.reverse()
    return path

def A_star(g: Graph, source: Node, target: Node) -> list[int]:
    if not g.nodes:
        raise ValueError('Graph is empty')
    if source.id not in g.nodes:
        raise ValueError(f'source node with id: {source.id} not in graph')
    if target.id not in g.nodes:
        raise ValueError(f'target node with id: {target.id} not in graph')

    INF = float('inf')
    source_id = source.id
    target_id = target.id
    minHeap = heapdict()
    prev = {}
    gScore = {source_id : 0} # g(n): sum of weights from source to curr
    minHeap[source_id] = 0

    while minHeap:
        curr_id, _ = minHeap.popitem()

        if curr_id == target_id:
            return construct_path(prev, target_id)

        for neighbour in g.adj_list.get(curr_id, []):
            edge_len = min([edge.length for edge in g.edges.get((curr_id, neighbour), [])]) # sometimes multiple edges, we will always choose the min(length) edge
            if neighbour not in gScore:
                gScore[neighbour] = INF
            tentative_gScore = gScore[curr_id] + edge_len
            if tentative_gScore < gScore[neighbour]:
                gScore[neighbour] = tentative_gScore
                minHeap[neighbour] = tentative_gScore + h(g.nodes[neighbour], target) #f(n) = g(n) + h(n)
                prev[neighbour] = curr_id

    return []

# if finding shortest path from source to every node t in targets
def all_multi_targets_A_star(g: Graph, source: Node, targets: list[Node]) -> list[list[int]]:
    paths = []
    for t in targets:
        path = A_star(g, source, t)
        paths.append(path)
    return paths

In [ ]:
# if finding shortest path from source to closest target in targets
# will optimise this and implement remove node and remove edge in Graph later
def add_dummy_node_and_reverse_graph(g: Graph, targets: list[Node]) -> Graph:
    new_graph = deepcopy(g)
    dummy_node = Node(
        id = 0,
        x_coord = 0,
        y_coord = 0,
        primary = False
    )
    new_graph.add_node(dummy_node)
    
    for t in targets:
        edge = Edge(
            node_id_from = 0,
            node_id_to = t.id,
            length = 0,
            name = ['dummy']
        )
        new_graph.add_edge(edge)

    new_graph = reverse_graph(new_graph)
    return (new_graph, dummy_node)
  
def single_multi_target_A_star(g: Graph, source: Node, targets: list[Node]) -> list[int]:
    (new_graph, dummy_node) = add_dummy_node_and_reverse_graph(g)
    return A_star(new_graph, dummy_node, source)

In [ ]:
def shortest_paths_to_charging_stations_A_star(g: Graph, V: list[Node]) -> list[tuple[int, int, list[int]]]: #(u_id, v_id, path)
    shortest_paths = []
    for v in V:
        for u in g.nodes.values():
            path = A_star(g, u, v)
            shortest_paths.append((u.id, v.id, path))
    return shortest_paths

PRIMARY_NODES = [node for node in graph.nodes.values() if node.primary]
sample_primary = random.sample(PRIMARY_NODES, 5)

charging_stations_nodes = [graph.nodes.get(node_id, None) for node_id in charging_stations]
start_time = time.time()
for _, u in graph.nodes.items():
    all_multi_targets_A_star(graph, u, sample_primary)
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)

942.4815828800201


Hér er mjög mikill munur á tímaum sem það tekur fyrir Dijkstra og A* að finna vegalengdir frá öllum hnútum u til $v_i$ í netinu þar sem að A star er single optimised fyrir single source target node en dijsktra fyrir alla punkta blablablablalbabl

***2.3.6 Staðsetning hleðslustöðva (⋆⋆)***

Ef við setjum $k$ hleðslustöðvar í hnúta $v_1, . . . , v_k$ þá látum við markfallið vera:
$$F(v_1, ... , v_k) \sum_{v \in V} \min_{i=1, ... ,k} d(u, v_i)$$
þ.e. fyrir hvern hnút í netinu reiknum við stystu fjarlægð frá honum til næstu hleðslustöðvar
og leggjum saman yfir alla hnúta í netinu. Finnið bestu lausn fyrir k = 1, með því að prófa alla hnúta sem hægt er að setja
hleðslustöð í og veljið þann sem gefur minnsta markfall. Athugið að eingöngu þeir hnútar
sem eru merktir sem primary geta verið hleðslustöðvar.

In [ ]:
PRIMARY_NODES = [node for node in graph.nodes.values() if node.primary]

def objective_function(g: Graph, V: list[Node]) -> list[tuple[float, int]]:
    reversed_graph = reverse_graph(g)
    sums = []
    INF = float('inf')
    for v in V:
        (dist, _) = Dijkstra(reversed_graph, v.id)
        curr_sum = sum(d for d in dist.values() if d != INF) # Unreachable if d == inf
        sums.append((curr_sum, v.id))
    return min(sums, key=lambda x: x[0])

objective_function(graph, PRIMARY_NODES)


(0, 5541083892)

***2.3.7 Gráðug reiknirit (⋆⋆)***

Útfærið gráðugt reiknirit sem leitar að bestu lausn fyrir $k = 2, . . . , 10$ með því að leysa
vandamálið fyrir $k−1$ hleðslustöðvum og bæta þá við þann hnút sem gefur minnsta markfall,
miðað við að ekki sé hægt að breyta $v_1, . . . , v_{k−1}$.
Sýnið á korti hvaða hleðslustöðvar eru valdar fyrir $k = 10$ og mælið tímann sem reikniritið
tekur.

In [113]:
def greedy(k: int, g: Graph, V: list[Node]) -> set[int]:
    # Solve for k = 1 and cache results
    reversed_graph = reverse_graph(graph)
    sums = []
    INF = float('inf')
    results = {}
    for v in V:
        (dist, _) = Dijkstra(reversed_graph, v.id)
        results[v.id] = dist
        curr_sum = sum(d for d in dist.values() if d != INF)
        sums.append((curr_sum, v.id))
    v_1 = min(sums, key=lambda x: x[0])[1]
    V_k = {v_1}

    for _ in range(k - 1):
        (min_sum, v_i) = (INF, None)
        for v in V:
            if v.id in V_k:
                continue

            curr_sum = 0
            for u in g.nodes:
                candidates_dists = [
                        results[v_k].get(u, INF)
                        for v_k in V_k | {v.id}
                        if results[v_k][u] != INF
                    ]
                if candidates_dists:
                    curr_min = min(candidates_dists)
                    curr_sum += curr_min
                
            if curr_sum < min_sum:
                (min_sum, v_i) = (curr_sum, v.id)
            
        V_k.add(v_i)
    
    return V_k

In [114]:
sol = greedy(10, graph, PRIMARY_NODES)
print(sol)

{253702373, 62975525, 470316424, 1204996745, 252165232, 2948755314, 4159611763, 5541083892, 470320635, 34827739}


***2.3.8 Skárri gráðug reiknirit (⋆⋆)***

Gráðuga reikniritið á það til að mála sig út í horn með því að velja lélegan fyrsta hnút.
Breytið leitinni þannig að þið veljið handahófskenndan fyrsta hnút og farið endurkvæmt í
tilfellin $k = 2, . . . , 10.$ Í hverju undirtilfelli finnið þið 2 bestu hnútana sem koma til greina
en eru langt frá hvor öðrum og prófið endurkvæmt alla möguleika. Haldið utan um bestu
lausnina sem finnst fyrir nokkur hanndahófskennda upphafspunkta og sýnið bestu lausn á
korti. Hve mikinn tíma tekur reikniritið ykkar?

In [ ]:
def better_greedy(k: int, g: Graph, V: list[Node]) -> set[tuple[int, float]]:
    def helper(V_k: set[int], min_score: float) -> tuple[set[int], float]: # (V_k, score)
        if len(V_k) == k:
            return (V_k, min_score)

        (min_i_score, min_v_i) = (INF, None)
        (min_j_score, min_j_i) = (INF, None)
        for i in range(n):
            v_i = V[i]
            v_i_id = v_i.id
            for j in range(i + 1, n):
                v_j = V[j]
                v_j_id = v_j.id
                if v_i_id in V_k or v_j_id in V_k:
                    continue
            
            curr_i_score = 0
            curr_j_score = 0
            curr_scalar = (0.5 * h(v_i, v_j))
            for u in g.nodes:
                candidates_i_scores = [
                    results[v_k].get(u, INF) - curr_scalar # We deduct by this scalar because we also care about the distance between the nodes
                    for v_k in V_k | {v_i_id}
                    if results[v_k][u] != INF
                ]

                candidates_j_scores = [
                    results[v_k].get(u, INF) - curr_scalar
                    for v_k in V_k | {v_j_id}
                    if results[v_k][u] != INF
                ]

                if candidates_i_scores and candidates_j_scores:
                    curr_i_min = min(candidates_i_scores)
                    curr_i_score += curr_i_min
                    curr_j_min = min(candidates_j_scores)
                    curr_j_score += curr_j_min
                
            if curr_i_score < min_i_score:
                (min_i_score, min_v_i) = (curr_i_score, v_i_id)
            
            if curr_j_score < min_j_score:
                (min_j_score, min_j_i) = (curr_j_score, v_j_id)
                
        return min(helper(V_k | {min_v_i}, min_i_score), helper(V_k | {min_j_i}, min_j_score), key = lambda x: x[1])

    reversed_graph = reverse_graph(graph)
    n = len(V)
    INF = float('inf')
    results = {}
    for v in V:
        (dist, _) = Dijkstra(reversed_graph, v.id)
        results[v.id] = dist
    
    first = random.choice(V).id
    return helper({first}, 0) #{(V_k, total_score)}

In [125]:
solution = better_greedy(3, graph, PRIMARY_NODES)
print(solution)

KeyboardInterrupt: 